## Agentic API 101

This document talks about the Agentic APIs in Llama Stack. Before you begin, please ensure Llama Stack is installed and set up by following the [Getting Started Guide](https://llama-stack.readthedocs.io/en/latest/getting_started/index.html).

Starting Llama 3.1 you can build agentic applications capable of:

- breaking a task down and performing multi-step reasoning.
- using tools to perform some actions
  - built-in: the model has built-in knowledge of tools like search or code interpreter
  - zero-shot: the model can learn to call tools using previously unseen, in-context tool definitions
- providing system level safety protections using models like Llama Guard.

An agentic app requires a few components:
- ability to run inference on the underlying Llama series of models
- ability to run safety checks using the Llama Guard series of models
- ability to execute tools, including a code execution environment, and loop using the model's multi-step reasoning process

All of these components are now offered by a single Llama Stack Distribution. Llama Stack defines and standardizes these components and many others that are needed to make building Generative AI applications smoother. Various implementations of these APIs are then assembled together via a **Llama Stack Distribution**.



### Run Agent example

Please check out examples with client SDKs to talk with the Llama Stack server in our [llama-stack-apps](https://github.com/meta-llama/llama-stack-apps) repo. 

In this tutorial, with the `Llama3.1-8B-Instruct` server running, we can use the following code to run a simple agent example:

Set up your connection parameters:

In [1]:
HOST = "localhost"  # Replace with your host
PORT = 8321  # Replace with your port
# MODEL_NAME = "meta-llama/Llama-3.2-3B-Instruct"
MODEL_NAME="deepseek-ai/DeepSeek-R1-Distill-Qwen-14B"


In [2]:
import os

from dotenv import load_dotenv

# load_dotenv()
# BRAVE_SEARCH_API_KEY = os.environ["BRAVE_SEARCH_API_KEY"]


In [3]:
import os

from llama_stack_client import LlamaStackClient
from llama_stack_client.lib.agents.agent import Agent
from llama_stack_client.lib.agents.event_logger import EventLogger


async def agent_example():
    client = LlamaStackClient(base_url=f"http://{HOST}:{PORT}")
    agent = Agent(
        client,
        model=MODEL_NAME,
        instructions="You are a helpful assistant! If you call builtin tools like brave search, follow the syntax brave_search.call(…)",
        sampling_params={
            "strategy": {
                "type": "greedy",
            },
        },
        tools=[
            {
                "type": "brave_search",
                "engine": "brave",
                "api_key": BRAVE_SEARCH_API_KEY,
            }
        ],
    )
    session_id = agent.create_session("test-session")
    print(f"Created session_id={session_id} for Agent({agent.agent_id})")

    user_prompts = [
        "I am planning a trip to Switzerland, what are the top 3 places to visit?",
        "What is so special about #1?",
    ]

    for prompt in user_prompts:
        response = agent.create_turn(
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
            session_id=session_id,
        )

        async for log in EventLogger().log(response):
            log.print()


await agent_example()


NameError: name 'BRAVE_SEARCH_API_KEY' is not defined

We have come a long way from getting started to understanding the internals of Llama-Stack! 

Thanks for joining us on this journey. If you have questions-please feel free to open an issue. Looking forward to what you build with Open Source AI!